In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
from glob import glob
from concurrent.futures import ProcessPoolExecutor, as_completed

import numpy as np
import pandas as pd

import settings as conf
import metadata
from utils import is_number, chunker
from results.multixcan import MXPhenoInfo, MXPhenoResults

# fastENLOC reading functions

In [ ]:
with open(conf.FASTENLOC_GTEX_TISSUES_FILE, 'r') as f:
    FASTENLOC_TISSUES_NAMES = set([x.strip() for x in f.readlines()])

In [ ]:
# create summary of files
_path = os.path.join(conf.FASTENLOC_RESULTS_DIR['RapidGWASProject'], '*.enloc.sig.out.gz')
display(_path)
all_fastenloc_results_files = glob(_path)

In [ ]:
PHENO_CODES = set(metadata.RAPID_GWAS_PHENO_INFO.index)

In [ ]:
len(PHENO_CODES)

In [ ]:
phenos = []
tissues = []

for f in all_fastenloc_results_files:
    split = f.split('-')
    
    tiss = split[-1].split('.')[0]
    if tiss not in FASTENLOC_TISSUES_NAMES:
        tiss = '-'.join(split[-2:]).split('.')[0]
    
    phen = split[1]
    if phen not in PHENO_CODES:
        phen = '-'.join(split[1:3])
    
    phenos.append(phen)
    tissues.append(tiss)

results_summary = pd.DataFrame({'pheno': phenos, 'tissue': tissues, 'file': all_fastenloc_results_files})

In [ ]:
display(results_summary.shape)
assert results_summary.shape[0] == int(conf.FASTENLOC_EXPECTED_PHENOTYPES['RapidGWASProject'] * conf.GTEX_MODELS_N_EXPECTED_TISSUES)

In [ ]:
results_summary.head()

In [ ]:
results_summary['pheno'].unique().shape

In [ ]:
assert np.all([t in PHENO_CODES for t in results_summary['pheno'].unique()])

In [ ]:
assert results_summary['tissue'].unique().shape[0] == 49

In [ ]:
assert np.all([t in FASTENLOC_TISSUES_NAMES for t in results_summary['tissue'].unique()])

In [ ]:
_all_tissues_in_results = results_summary['tissue'].unique()
assert np.all([t in _all_tissues_in_results for t in FASTENLOC_TISSUES_NAMES])

In [ ]:
assert len(all_fastenloc_results_files) == int(conf.SMULTIXCAN_EXPECTED_PHENOTYPES['RapidGWASProject'] * conf.GTEX_MODELS_N_EXPECTED_TISSUES)

In [ ]:
def read_fe(filename): 
    fe_data = pd.read_csv(filename, sep='\s+', usecols=[0, 5], header=None).rename(columns={0: 'gene_cluster', 5: 'rcp'})
    fe_data = fe_data = fe_data.assign(gene=fe_data['gene_cluster'].apply(lambda x: x.split(':')[0])) 
    fe_data = fe_data.assign(gene_cluster=fe_data['gene_cluster'].apply(lambda x: x.split(':')[1]))
    
    fe_max = fe_data.groupby('gene')['rcp'].sum()
    return fe_max

def read_pheno(pheno): 
    _tmp = results_summary[results_summary['pheno'] == pheno] 
    if _tmp.shape[0] == 0: 
        return None 
     
    _res = {} 
    _all_genes = set() 
    for idx, _data in _tmp.iterrows(): 
        tissue_data = read_fe(_data.file) 
        _res[_data.tissue] = tissue_data 
        _all_genes.update(tissue_data.index) 
    
    return pd.DataFrame(_res, index=_all_genes).fillna(-1).max(axis=1)

### Testing

In [ ]:
# testing
t = read_fe(os.path.join(conf.FASTENLOC_RESULTS_DIR['RapidGWASProject'], 'fastenloc-J15-Whole_Blood.enloc.sig.out.gz'))

In [ ]:
t.sort_values(ascending=False).head()

In [ ]:
_gid = metadata.GENE_NAME_TO_ID_MAP['SLCO4C1']
assert t[_gid] == (1.869e-07 + 1.578e-11 + 5.201e-12 + 1.409e-13 + 2.694e-11)

In [ ]:
_gid = metadata.GENE_NAME_TO_ID_MAP['RP11-34P13.7']
assert t[_gid] == 0.00, t[_gid]

```parallel -j4 'zcat {} | grep ENSG00000049246' ::: fastenloc-1180-*.gz | column -t | sort -k6 -g```

In [ ]:
t = read_pheno('J15')

In [ ]:
t.sort_values(ascending=False).head()

In [ ]:
t.loc['ENSG00000116985']

In [ ]:
assert t.loc['ENSG00000116985'] == (1.124e-02 + 5.425e-07 + 1.153e-08 + 2.864e-06 + 1.644e-06)

# ENLOC reading functions

In [ ]:
display(conf.FASTENLOC_RESULTS_DIR['GTEX_GWAS'])

In [ ]:
def read_enloc(enloc_result_path):
    enloc_filename = enloc_result_path.split('/')[-1]
    phenotype = enloc_filename.split('__PM__')[0]
    tissue = enloc_filename.split('__PM__')[1].split('.enloc.rst.gz')[0]
    
    enloc_result = pd.read_csv(enloc_result_path, sep='\s+')
    enloc_result = enloc_result.assign(gene_id=enloc_result['molecular_qtl_trait'].apply(lambda x: x.split('.')[0]))
    
    return enloc_result.groupby('gene_id')['locus_rcp'].max().rename(tissue)

def read_enloc_pheno(phenotype_code):
    all_tissues_data = {}
    all_genes = set()
    
    all_phenotype_files = glob(os.path.join(conf.FASTENLOC_RESULTS_DIR['GTEX_GWAS'], f'{phenotype_code}__PM__*.enloc.rst.gz'))
    
    for f in all_phenotype_files:
        f_data = read_enloc(f)
        all_tissues_data[f] = f_data
        all_genes.update(f_data.index)
    
    return pd.DataFrame(all_tissues_data, index=all_genes).fillna(-1).max(axis=1)

In [ ]:
t = read_enloc_pheno('SSGAC_Education_Years_Pooled')

In [ ]:
t.head()

In [ ]:
# testing
assert t.loc['ENSG00000106113'] == 0.00
assert t.loc['ENSG00000164050'] == 0.006
assert t.loc['ENSG00000081377'] == 0.317

# Get Rapid GWAS phenotypes

In [ ]:
all_smultixcan_files = glob(os.path.join(conf.SMULTIXCAN_RESULTS_DIR['RapidGWASProject'], '*.tsv.gz'))
all_smultixcan_phenotypes = [MXPhenoResults(p) for p in all_smultixcan_files]

In [ ]:
assert len(all_smultixcan_files) == len(all_smultixcan_phenotypes) == conf.SMULTIXCAN_EXPECTED_PHENOTYPES['RapidGWASProject']

In [ ]:
all_smultixcan_files[:5]

# Get GTEx GWAS phenotypes

In [ ]:
_path = os.path.join(conf.SMULTIXCAN_RESULTS_DIR['GTEX_GWAS'], '*_ccn30.txt')
display(_path)
all_extra_results_files = glob(_path)
assert len(all_extra_results_files) == conf.SMULTIXCAN_EXPECTED_PHENOTYPES['GTEX_GWAS']

In [ ]:
all_extra_results_files[:5]

In [ ]:
_file_pattern = '(?P<code>[^/]+)_smultixcan_imputed_gwas_gtexv8mashr_ccn30\.txt'
all_extra_phenotypes = [MXPhenoResults(p, _file_pattern) for p in all_extra_results_files]
all_extra_phenotypes_plain_names = pd.Index([p.pheno_info.get_plain_name() for p in all_extra_phenotypes])

display(len(all_extra_phenotypes))
assert len(all_extra_phenotypes) == conf.SMULTIXCAN_EXPECTED_PHENOTYPES['GTEX_GWAS']

In [ ]:
all_extra_phenotypes[0].pheno_info.get_plain_name()

In [ ]:
all_extra_phenotypes_codes = [os.path.basename(f).split('_smultixcan_imputed_gwas_gtexv8mashr_ccn30')[0] for f in all_extra_results_files]
assert len(all_extra_phenotypes_codes) == conf.SMULTIXCAN_EXPECTED_PHENOTYPES['GTEX_GWAS']

In [ ]:
all_extra_phenotypes_codes[:5]

In [ ]:
all_extra_phenotypes_codes = set(all_extra_phenotypes_codes)

# Read all results

In [ ]:
def read_generic(pheno_code):
    if pheno_code in all_extra_phenotypes_codes:
        return read_enloc_pheno(pheno_code)
    else:
        return read_pheno(pheno_code)

def _get_combined_results(phenos):
    return {pheno.pheno_info.get_plain_name() : read_generic(pheno.pheno_info.pheno_code) for pheno in phenos}

In [ ]:
# testing
_tmp = _get_combined_results(all_smultixcan_phenotypes[:4])
assert len(_tmp) == 4

In [ ]:
# testing
_tmp = _get_combined_results(all_extra_phenotypes[:4])
assert len(_tmp) == 4

In [ ]:
_pending = read_generic('C_MYELOID-LEUKAEMIA')

In [ ]:
_pending.head()

In [ ]:
_pending = read_generic('SSGAC_Education_Years_Pooled')

In [ ]:
_pending.head()

In [ ]:
def _run_all(phenotype_chunks, n_jobs=conf.N_JOBS_HIGH):
    all_results = {}
    
    with ProcessPoolExecutor(max_workers=n_jobs) as executor:
        tasks = [executor.submit(_get_combined_results, chunk) for chunk in phenotype_chunks]
        for future in as_completed(tasks):
            res = future.result()
            all_results.update(res)

    return all_results

In [ ]:
# phenotype_chunks = chunker(all_smultixcan_phenotypes[:5] + all_extra_phenotypes[:5], 2)
phenotype_chunks = chunker(all_smultixcan_phenotypes + all_extra_phenotypes, 200)

In [ ]:
all_results = _run_all(phenotype_chunks)

## Save as DataFrame

In [ ]:
assert len(all_results) == conf.FASTENLOC_EXPECTED_PHENOTYPES['RapidGWASProject'] + conf.FASTENLOC_EXPECTED_PHENOTYPES['GTEX_GWAS']

In [ ]:
fastenloc_genes_associations = pd.DataFrame(all_results)
fastenloc_genes_associations.index.rename('gene_id', inplace=True)

assert fastenloc_genes_associations.index.is_unique

display(fastenloc_genes_associations.shape)
display(fastenloc_genes_associations.head())

In [ ]:
# check genes id format
_tmp = pd.Series(fastenloc_genes_associations.index).apply(len).value_counts()
display(_tmp)
assert _tmp.shape[0] == 1
assert _tmp.index[0] == 15

In [ ]:
# check columns data type
_tmp = fastenloc_genes_associations.dtypes.value_counts()
display(_tmp)
assert _tmp.shape[0] == 1

In [ ]:
fastenloc_genes_associations.isna().sum().sum()

In [ ]:
fastenloc_genes_associations.dropna(axis=0, how='all').shape

In [ ]:
display(fastenloc_genes_associations.shape)

In [ ]:
assert fastenloc_genes_associations.shape == (37967, conf.FASTENLOC_EXPECTED_PHENOTYPES['RapidGWASProject'] + conf.FASTENLOC_EXPECTED_PHENOTYPES['GTEX_GWAS'])

In [ ]:
# some testing
t = fastenloc_genes_associations['SSGAC_Education_Years_Pooled']

In [ ]:
t.head()

In [ ]:
assert t.loc['ENSG00000106113'] == 0.00
assert t.loc['ENSG00000164050'] == 0.006
assert t.loc['ENSG00000081377'] == 0.317
assert pd.isnull(t.loc['ENSG00000000938'])

### Save

In [ ]:
# Save
fastenloc_genes_associations_filename = os.path.join(conf.GENE_ASSOC_DIR, f'fastenloc-torus-rcp.pkl.xz')
display(fastenloc_genes_associations_filename)

In [ ]:
fastenloc_genes_associations.to_pickle(fastenloc_genes_associations_filename)

### Save for publication

In [ ]:
output_file = os.path.join(conf.GENE_ASSOC_DIR, 'fastenloc-torus-rcp.tsv.gz')
display(output_file)

In [ ]:
fastenloc_genes_associations.to_csv(output_file, sep='\t', float_format='%.4e')

In [ ]:
# test "for publication" file
_tmp = pd.read_csv(output_file, sep='\t', index_col='gene_name')

In [ ]:
display(_tmp.shape)
assert _tmp.shape == fastenloc_genes_associations.shape

In [ ]:
_tmp.head()

In [ ]:
assert not _tmp.isin([np.inf, -np.inf]).any().any()

In [ ]:
assert _tmp.isna().any().any()

In [ ]:
_tmp_flat = pd.Series(_tmp.values.flatten()).dropna()
assert ((_tmp_flat >= 0) & (_tmp_flat <= 3)).all().all()

In [ ]:
assert np.allclose(_tmp.values, fastenloc_genes_associations.values, atol=1e-320, rtol=1e-4, equal_nan=True)

### Save in HDF5 format for webapp

In [4]:
fastenloc_gene_associations = pd.read_pickle('/mnt/phenomexcan_base/gene_assoc/fastenloc-torus-rcp.pkl.xz')

In [5]:
fastenloc_gene_associations.shape

(37967, 4091)

In [6]:
from utils import simplify_string_for_hdf5

In [7]:
FASTENLOC_HDF5_FILE = os.path.join(conf.GENE_ASSOC_DIR, 'fastenloc-torus-rcp.h5')
display(FASTENLOC_HDF5_FILE)

'/mnt/phenomexcan_base/gene_assoc/fastenloc-torus-rcp.h5'

In [8]:
with pd.HDFStore(FASTENLOC_HDF5_FILE, mode='w', complevel=1) as store:
    for col in fastenloc_gene_associations.columns:
        print('.', flush=True, end='')
        
        clean_col = simplify_string_for_hdf5(col)
        store[clean_col] = fastenloc_gene_associations[col]

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [13]:
# testing
with pd.HDFStore(FASTENLOC_HDF5_FILE, mode='r') as store:
    store_keys = list(store.keys())
    assert len(store_keys) == fastenloc_gene_associations.shape[1]
    display(store_keys[:5])
    
    clean_col = simplify_string_for_hdf5('100001_raw-Food_weight')
    data = store[clean_col]
    assert data.shape == (37967,), data.shape
    assert data.loc['ENSG00000267462'] == 3.6406000000000003e-06

['/c100001_raw_Food_weight',
 '/c100002_raw_Energy',
 '/c100003_raw_Protein',
 '/c100004_raw_Fat',
 '/c100005_raw_Carbohydrate']